In [15]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('AOAI_API_KEY')
openai.api_type = "azure"
openai.api_base = os.getenv('AOAI_ENDPOINT')
openai.api_version = "2023-05-15"

def get_completion_from_messages(messages,temperature=0):
    response = openai.ChatCompletion.create(
        engine=os.getenv('AOAI_MODEL'),
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )

    print(response.usage)

    return response.choices[0].message["content"]

In [21]:
# CSVファイルの読み込み
import pandas as pd
df = pd.read_csv('../private/testdata.csv', encoding='utf-8')

# 件名と本文と差出人 : (住所)のみに絞る
df = df[['件名', '本文', '差出人 : (住所)']]
#df.head()

# 2行目の"本文”を取得
text = df.iloc[1,1]

# textから2つ続く改行を削除
text = text.replace("\r\n", "\n")
text = text.replace("\n\n", "\n")

# textから2個以上多く続く空白を1つの空白に置換
text = text.replace("  ", " ")

text


'こんにちは。洗濯機が壊れたことに関する相談、ありがとうございました。 \n洗濯機が壊れてしまったことは大変申し訳ありません。 保証期間内ですので、無償で修理させていただきます。 \n修理の手順は以下の通りです。\n1.\tお客様のお住まいの地域にある修理センターにお電話ください。電話番号は以下のリンクからご確認いただけます。\n2.\t修理センターの担当者が、洗濯機の型番と故障状況をお伺いします。また、修理日時と修理費用（無料）をお知らせします。\n3.\t修理日時に合わせて、修理センターから派遣された技術者がお客様のご自宅に伺います。技術者が洗濯機を点検し、修理します。\n4.\t修理が完了したら、技術者が動作確認を行います。また、修理内容と保証書をお渡しします。\n5.\t修理後も万が一不具合が発生した場合は、再度修理センターにご連絡ください。迅速に対応させていただきます。\n\n以上が修理の手順です。ご不明な点がありましたら、お気軽にお問い合わせください。 ご迷惑をおかけして申し訳ありませんが、何卒よろしくお願いします。\n2023年6月22日(木) 12:19 Shuji Yamaguchi (CSA DEV ADVOCATE) <Shuji.Yamaguchi@microsoft.com <mailto:Shuji.Yamaguchi@microsoft.com> >:\n\n\tこんにちは。先日購入した洗濯機が壊れてしまいました。\n\t洗濯中に異音がして止まってしまい、再起動もできませんでした。\n\t保証期間内ですが、修理にはどうすればいいでしょうか？ \n\t返信をお待ちしています。 よろしくお願いします。\n'

In [26]:
import tiktoken

encoding_35 = tiktoken.encoding_for_model("gpt-3.5-turbo-0301")

def calc_token_tiktoken(text):
    encoding = tiktoken.get_encoding(encoding_35.name)
    num_tokens = len(encoding.encode(text))
    return num_tokens


In [23]:
def getAnalyedMailContent(text):
    system_prompt = """あなたは日本語テキスト文章解析器です。
入力されるメールのやりとりに対して、文章を読取、以下のルールで文章を分割します。
なお、分割した結果は、以下の出力フォーマットに従ってJSONのみ回答してください。
*説明や補足などは不要です。JSONのみ回答してください。

ルール:
 - 差出人が変わる場所で、文章を分割します。
 - 最後のメールが途中で切れている場合は、そのメールの文章は余りとして出力します。

出力フォーマット:
{
    paragraphs: ["メール1","メール2"],
    surplus: "メール3途中まで"
}
"""

    user_prompt = text

    messages =  [
    {'role':'system', 'content':system_prompt},
    {'role':'user', 'content':user_prompt},
    ]
    response = get_completion_from_messages(messages, temperature=1)

    #平文のresponseをJSONに変換
    import json
    response_json = json.loads(response)
    return response_json


In [28]:
print(calc_token_tiktoken(text))
res = getAnalyedMailContent(text)

625


{
  "completion_tokens": 599,
  "prompt_tokens": 841,
  "total_tokens": 1440
}


In [29]:
res

{'paragraphs': ['こんにちは。洗濯機が壊れたことに関する相談、ありがとうございました。 \n洗濯機が壊れてしまったことは大変申し訳ありません。 保証期間内ですので、無償で修理させていただきます。 \n修理の手順は以下の通りです。\n1.\tお客様のお住まいの地域にある修理センターにお電話ください。電話番号は以下のリンクからご確認いただけます。\n2.\t修理センターの担当者が、洗濯機の型番と故障状況をお伺いします。また、修理日時と修理費用（無料）をお知らせします。\n3.\t修理日時に合わせて、修理センターから派遣された技術者がお客様のご自宅に伺います。技術者が洗濯機を点検し、修理します。\n4.\t修理が完了したら、技術者が動作確認を行います。また、修理内容と保証書をお渡しします。\n5.\t修理後も万が一不具合が発生した場合は、再度修理センターにご連絡ください。迅速に対応させていただきます。\n\n以上が修理の手順です。ご不明な点がありましたら、お気軽にお問い合わせください。 ご迷惑をおかけして申し訳ありませんが、何卒よろしくお願いします。',
  'こんにちは。先日購入した洗濯機が壊れてしまいました。\n洗濯中に異音がして止まってしまい、再起動もできませんでした。\n保証期間内ですが、修理にはどうすればいいでしょうか？ \n返信をお待ちしています。 よろしくお願いします。'],
 'surplus': ''}